In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np#
import copy
from emlp.solver.representation import *
from emlp.solver.groups import *
from emlp.models.mlp import uniform_rep
import unittest
from jax import vmap
import jax.numpy as jnp
import logging
import argparse
import sys
import copy
from functools import partialmethod
import logging
logging.getLogger().setLevel(logging.INFO)
import jax
from emlp.solver.linear_operators import lazify

In [2]:
W = V(S(3))
R =2*W+3*W**2

In [3]:
P = (R>>R).equivariant_projector()

INFO:root:V² cache miss
INFO:root:Solving basis for V², for G=S(3)
INFO:root:V³ cache miss
INFO:root:Solving basis for V³, for G=S(3)
INFO:root:V⁴ cache miss
INFO:root:Solving basis for V⁴, for G=S(3)


In [4]:
Q = (R>>R).equivariant_basis()

In [5]:
Q = Q@jnp.eye(Q.shape[-1])

In [6]:
jnp.linalg.norm((Q@Q.H).to_dense()-(Q@Q.H).to_dense().T)

AttributeError: '_DeviceArray' object has no attribute 'H'

In [5]:
import torch
from emlp.solver.pytorch_port import torchify_fn, to_jax,to_pytorch

In [6]:
v = torch.randn(P.shape[-1],requires_grad=True).cuda()
w  =torch.randn(P.shape[-1]).cuda()

In [9]:
jnp.linalg.norm(P.to_dense()-P.to_dense().T)

DeviceArray(0., dtype=float32)

In [10]:
s = (P.to_dense()-P.to_dense().T)
jnp.linalg.norm(s)

DeviceArray(0., dtype=float32)

In [6]:
jnp.abs(s).sum()

DeviceArray(0., dtype=float32)

In [7]:
proj = lambda v: P@v
projt = torchify_fn(proj)

In [8]:
L = (w@(projt(v)))
#L = w@projt(v)
jw = to_jax(w)
L2 = lambda v: jw@(P@v)

In [15]:
P_dense = P@jnp.eye(P.shape[-1])

In [9]:
jax.grad(L2)(to_jax(v))[:10]

DeviceArray([-0.10575901,  0.27642417,  0.27642426, -1.218089  ,
             -1.2420236 , -1.242024  ,  0.9540143 , -0.5370708 ,
             -0.5370711 ,  0.78159   ], dtype=float32)

In [150]:
(P_dense@jw[:,None])[:10,0]

DeviceArray([ 0.13978752,  0.6574347 ,  0.65743494, -0.6165879 ,
             -0.06620419, -0.06620425,  0.6574346 ,  0.13978744,
              0.65743464, -0.06620422], dtype=float32)

In [151]:
jnp.linalg.norm(P_dense-P_dense.T)

DeviceArray(3.3333337, dtype=float32)

In [16]:
P_dense - P_dense.T

DeviceArray([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [12]:
g_true = projt(w)

In [10]:
g = torch.autograd.grad(L,v)[0]

In [11]:
g

tensor([-0.1058,  0.2764,  0.2764,  ..., -0.3449, -0.3449,  0.7111],
       device='cuda:0')

In [13]:
g_true

tensor([-0.1058,  0.2764,  0.2764,  ..., -0.3449, -0.3449,  0.7111],
       device='cuda:0')

In [70]:
(g-g_true).norm()/(g+g_true).norm()

tensor(0.9644, device='cuda:0')